# Intel One APi

In [1]:
%%writefile lab/hello.cpp
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <opencv2/core.hpp>
#include <opencv2/imgcodecs.hpp>
#include <opencv2/highgui.hpp>
#include <iostream>
#include <algorithm>
#include <CL/sycl.hpp>

using namespace cv;
using namespace std;
using namespace sycl;

int main(){
  string image_path = "p2.jpg";
    
  //# define queue which has default device associated for offload
  queue q;
  // for this exercise, we did not control what type of device we are offloading to,
  // so this output will tell us what we were assigned
  std::cout << "Device: " << q.get_device().get_info<info::device::name>() << std::endl;

  Mat img = imread(image_path, IMREAD_COLOR);
  if(img.empty()) {
    std::cout << "Could not read the image: " << image_path << std::endl;
    return 1;
  }

#if 0
  // the SYCL kernel, after the endif, will do the same as this C++ code would do
  for( int y = 0; y < img.rows; y++ ) {
    for( int x = 0; x < img.cols; x++ ) {
      for( int c = 0; c < img.channels(); c++ ) {
        img.at<Vec3b>(y,x)[c] =
          clamp( img.at<Vec3b>(y,x)[c] + 50, 0, 255 );
      }
    }
  }
#endif
  {
    CV_Assert(img.isContinuous());
    CV_CheckTypeEQ(img.type(), CV_8UC3, "");
    buffer<uint8_t, 3> frame_buffer(img.data, range<3>(img.rows, img.cols, 3));

    q.submit([&](handler& cgh) {
      auto pixels = frame_buffer.get_access<access::mode::read_write>(cgh);
      cgh.parallel_for(range<3>(img.rows, img.cols, 3), [=](item<3> item) {
        uint8_t p = pixels[item];
        pixels[item] = sycl::clamp(p+50,0,255);
      });
    });
    q.wait_and_throw();
  }

  imwrite("p2.png", img);

  return 0;
}

Overwriting lab/hello.cpp


***

### Build and Run
Select the cell below and click Run ▶ to compile and execute the code above:

In [3]:
! chmod 755 q; chmod 755 run_hello.sh;if [ -x "$(command -v qsub)" ]; then ./q run_hello.sh; else ./run_hello.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2063443.v-qsvr-1           ...ub-singleuser u179021         00:00:35 R jupyterhub     
2063620.v-qsvr-1           run_hello.sh     u179021                0 Q batch          

Waiting for Output ████████ Done⬇

########################################################################
#      Date:           Tue 29 Nov 2022 04:47:10 PM PST
#    Job ID:           2063620.v-qsvr-1.aidevcloud
#      User:           u179021
# Resources:           cput=35:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u179021 is compiling xpublog 1 -- Hello

_If the Jupyter cells are not responsive or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_

***

Wait for the above to be DONE before proceeding... or else the output image will not be ready to view!

## Review Image
The orginal image looks like this:

<img src="p2.jpg">

The output image looks like this (select this cell and click Run ▶ to refresh the image):

<img src="p2.png">

***